MongoDB with Python

In [99]:
from pymongo import MongoClient, errors, ASCENDING, DESCENDING

In [9]:
# init client to connect to mongoDB
my_client = MongoClient('mongodb://localhost:27017')

In [16]:
my_client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [24]:
# connect to db
my_client.list_database_names()

['SoundCloud', 'admin', 'auth_tutorial', 'config', 'local', 'store']

In [31]:
# connect to db store
db_store = my_client.store

In [26]:
db_store

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store')

In [40]:
# list collections from store db
print(db_store.list_collection_names())

['admin', 'users']


In [41]:
users_col = db_store.users

In [42]:
users_col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'users')

C - Create

In [36]:
users_col.insert_one({
    "name": "John",
    "age": 29
})

In [38]:
new_user = {
    "name": "Keanu Reeves",
    "age": 57
}

In [39]:
users_col.insert_one(new_user)

In [43]:
users_col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'users')

R - Read

In [46]:
users_col.find_one({"name":"Keanu Reeves"})

{'_id': ObjectId('668a85a873c8b630e2131b6a'),
 'name': 'Keanu Reeves',
 'age': 57}

In [47]:
users_col.find_one(new_user)

{'_id': ObjectId('668a85a873c8b630e2131b6a'),
 'name': 'Keanu Reeves',
 'age': 57}

U- Update

In [48]:
users_col.update_one({"name":"John"}, {"$set":{"name":"Will", "age":53}})

D - Delete

In [49]:
users_col.delete_one({"name":"Will"})

In [53]:
products_db = db_store.products

In [51]:
bag = {"name": "Bag", "price":23.78}

In [52]:
products_db.insert(bag)

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/1657952950.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  products_db.insert(bag)


ObjectId('668a889073c8b630e2131b6b')

In [55]:
phone = {"name":"iphone 11", "price": 800}
notebook = {"name":"Notebook", "price": 4.99}

In [56]:
products_db.insert_many([phone, notebook])

In [75]:
pen = {"name":"Pen2", "price":1.22}
insert_pen = products_db.insert_one(pen)

In [59]:
print(insert_pen.acknowledged)
print(insert_pen.inserted_id)

True
668a8a2173c8b630e2131b6f


In [60]:
water_bottle = {"name":"Water Bottle", "price": 14.5}
jar = {"name":"Jar", "price":4.99}
insert_several = products_db.insert_many([water_bottle, jar])
insert_several

In [61]:
print(insert_several.acknowledged)
print(insert_several.inserted_ids)


True
[ObjectId('668a8ac273c8b630e2131b70'), ObjectId('668a8ac273c8b630e2131b71')]


Error Handling When Inserting

In [67]:
try:
    products_db.insert_one(pen)
except errors.DuplicateKeyError:
    print("Duplicate Entry Detected")

Duplicate Entry Detected


In [69]:
try:
    products_db.insert_many([water_bottle, jar])
except errors.BulkWriteError:
    print("Bulk Error Detected")

Bulk Error Detected


In [70]:
mug = {"name":"Mug", "price":2.33}
try:
    products_db.insert_many([mug, jar])
except errors.BulkWriteError:
    print("Bulk Error Detected")

Bulk Error Detected


In [71]:
products_db

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'products')

Find Functionality

In [72]:
found_pen = products_db.find_one({"name":"Pen"})

In [73]:
found_pen["_id"]

ObjectId('668a89d773c8b630e2131b6e')

In [76]:
# cursor
products_db.find({"name":"Pen2"})

In [79]:
pen_cursor = products_db.find({"name":"Pen2"})
for pen in pen_cursor:
    print(pen["_id"])

668a8a2173c8b630e2131b6f
668a8e6173c8b630e2131b73


In [81]:
pen_cursor.alive #cursor went through all results

False

In [94]:
pen_cursor = products_db.find({"name":"Pen2"})
pen_cursor.alive #cursor has not gone through all results

True

In [95]:
pen_cursor.next()

{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}

Sorting

In [105]:
for document in products_db.find({}).sort("price", ASCENDING):
    print(document)

{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}


In [106]:
for document in products_db.find({}).sort("price", DESCENDING):
    print(document)

{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


In [101]:
ASCENDING

1

In [102]:
DESCENDING

-1

In [107]:
for document in products_db.find({}).sort("price", 1):
    print(document)

{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}


In [108]:
for document in products_db.find({}).sort([("price", ASCENDING),("name", ASCENDING)]):
    print(document)

{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}


In [109]:
for product in products_db.find({}).limit(5):
    print(product)

{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}


In [110]:
for product in products_db.find({}).skip(5):
    print(product)

{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


In [111]:
page_item_limit = 5
for page_num in [1, 2]:
    print("\n-- Page {} --".format(page_num))
    # Calculate the number of products to skip
    skip_amount = (page_num - 1) * page_item_limit
    # Find, skip, and limit the products for the current page
    for product in products_db.find({}).skip(skip_amount).limit(page_item_limit):
        print(product)


-- Page 1 --
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}

-- Page 2 --
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


Count documents

In [112]:
find_query_filter = {"name": "Pen2"}
find_cursor = products_db.find(find_query_filter)

In [113]:
find_cursor.count()

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/2083242281.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  find_cursor.count()


2

In [117]:
products_db.count_documents(find_query_filter)

2

In [118]:
find_cursor_skip = products_db.find(find_query_filter).skip(1)
products_db.count_documents(find_query_filter)

2

In [119]:
find_cursor_skip.count(with_limit_and_skip=True)

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/629907177.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  find_cursor_skip.count(with_limit_and_skip=True)


1

Distinct

In [121]:
product_cursor = products_db.find({})

In [124]:
distinct_count = product_cursor.distinct("name")
all_count = products_db.count_documents({})

print("{} unique product names found vs {} total entries".format(len(distinct_count), all_count))

8 unique product names found vs 9 total entries


Using projection

In [125]:
find_projection_cursor = products_db.find({"name":"Pen2"}, {"_id":0, "price":1})

In [126]:
for document in find_projection_cursor:
    print(document)

{'price': 1.22}
{'price': 1.22}


In [127]:
products_db.find_one({"name":"Pen2"}, {"_id":0, "price":1})

{'price': 1.22}